In [ ]:
import warnings
warnings.filterwarnings('ignore')

from wrantools.db import WMDBAdapter
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

Insert your credentials

In [ ]:
from wrantools.vault import get_vault_client
vault = get_vault_client()

The data is taken from the main data source of deployments, change data range

In [ ]:
sql = """
select depl.* ,
       sf2.upgrade_score as "Upgrade Score"
from tableau_deployments depl
left join (select wrike_account_id,
                  created_date,
                  industry,
                  upgrade_score,
                  row_number() over
                      (partition by wrike_account_id order by created_date desc) as rank
            from derived_sf2_account
            group by 1,2,3,4) sf2
    on depl."Account Number"=sf2.wrike_account_id and sf2.rank=1
where "Bill Date" BETWEEN '2018-01-01' AND '2019-10-31'
"""
db_client = WMDBAdapter(use_pandas=True)
df = db_client.pentaho.simple_get(sql)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
summary = df.describe(include='all')
summary = summary.transpose()
summary

In [ ]:
import pandas as pd
import numpy as np
import math as math
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import matplotlib

import matplotlib.pyplot as plt
from scipy import stats
matplotlib.style.use('ggplot')
%matplotlib inline

All fields where there are dates, transform to date type

In [ ]:
df['Bill Date']=pd.to_datetime(df['Bill Date'])
df['Became Paid Day']=pd.to_datetime(df['Became Paid Day'])
df['Start Date']=pd.to_datetime(df['Start Date'])
df['End Date']=pd.to_datetime(df['End Date'])

Make new fields for prediction model

In [ ]:
df['Age (Months)']=np.where(df['Became Paid Day']=="None",0,(df['Bill Date']-df['Became Paid Day']))
df['Age (Months)']=df['Age (Months)'] / pd.Timedelta(30, unit='d')
df['Age (Months)']=df['Age (Months)'][df['Age (Months)'].notnull()].apply(lambda x: int(x))
df['Age (Months)'].head()

In [ ]:
import time
from datetime import date
def ifnull(var, val):
    if pd.isnull(var):
        return val
    return var
df['Today']=pd.to_datetime(date.today())
# df['Duration Days']=np.where(pd.isnull(pd.to_datetime(df['End Date'])-pd.to_datetime(df['Start Date'])), 
#                              pd.to_datetime(df['Today'])-pd.to_datetime(df['Start Date']), 
#                              pd.to_datetime(df['End Date'])-pd.to_datetime(df['Start Date']))
df['Days from Today']=np.where(pd.isnull(pd.to_datetime(df['Today'])-pd.to_datetime(df['Start Date'])), 
                              pd.to_datetime(df['Today'])-pd.to_datetime(df['Bill Date']), 
                              pd.to_datetime(df['Today'])-pd.to_datetime(df['Start Date']))
df['Duration Days']=pd.to_datetime(df['End Date'])-pd.to_datetime(df['Start Date'])
#df['Duration Days']=np.where(pd.isnull(df['Duration Days']),
#                             pd.to_datetime(df['Today'])-pd.to_datetime(df['Bill Date']),df['Duration Days'])
df['Duration Days']=df['Duration Days'] / pd.Timedelta(1, unit='d')
df['Days from Today']=df['Days from Today'] / pd.Timedelta(1, unit='d')

In [ ]:
df['Package'].unique()

Make less deployment packages, unification of them

In [ ]:
df['Package (group)']=df['Package'].replace(['Large ','Expansion Large '],'Large')
df['Package (group)']=df['Package (group)'].replace(['Medium','26-50 User Deployment Package','Expansion Medium'],'Medium')
df['Package (group)']=df['Package (group)'].replace(['Expansion Small'],'Small')
df['Package (group)'].unique()

Conditions for overdue definition

In [ ]:
package_durations=df[df['Project Status'].isin(['Completed', 'Deemed Delivered'])].groupby(by=['Package (group)','Remote / Onsite'],as_index=False)['Duration Days'].median()

In [ ]:
package_durations.rename(columns={'Duration Days':'Package (group) durations'},inplace=True)

In [ ]:
package_durations

In [ ]:
df=pd.merge(df,package_durations,how='left', on=['Package (group)','Remote / Onsite'], sort=False)

In [ ]:
df['Overdue Days']=np.where(pd.isnull(df['Duration Days']),
                           df['Days from Today']-df['Package (group) durations'],
                           df['Duration Days']-df['Package (group) durations'])
df['Duration Overall']=np.where(pd.isnull(df['Duration Days']),
                                df['Days from Today'],
                                df['Duration Days'])

In [ ]:
sns.catplot(x="Package (group)", y="Overdue Days", col="Remote / Onsite", kind="box", data=df)

In [ ]:
df[df['Project Status'].isin(['Completed', 'Deemed Delivered'])].groupby(by=['Package (group)','Remote / Onsite'],as_index=False)['Overdue Days'].mean()

In [ ]:
conditions = [((df['Package (group)'] =='JumpStart') & (df['Duration Overall'] > 45))|
              ((df['Package (group)'] =='Small') & (df['Duration Overall'] > 60))|
              ((df['Package (group)'] =='Medium') & (df['Duration Overall'] > 75))|
              ((df['Package (group)'] =='Large') & (df['Duration Overall'] > 90))|
              ((df['Package (group)'] =='Custom Development') & (df['Duration Overall'] > 90))|
              ((df['Package (group)'] =='Custom Deployment') & (df['Duration Overall'] > 90))|
              ((df['Package (group)'] =='PS Pilot') & (df['Duration Overall'] > 90))|
              ((df['Package (group)'] =='Integrate') & (df['Duration Overall'] > 90))]
choices = ['T']
df['Overdue'] = np.select(conditions, choices, default='F')
df[['Package','Package (group)','Duration Overall','Duration Months','Overdue']].head()

In [ ]:
df['Duration Days'].hist()

In [ ]:
df['Duration Days'].median()

In [ ]:
df['Duration Overall'].hist()

In [ ]:
df[df['Duration Months']>6]['Bookings Amount'].sum()

In [ ]:
df.columns

In [ ]:
df['Project Status'].value_counts()

In [ ]:
df['Duration Months'].value_counts()

In [ ]:
df['Full Start Date'] = np.where(df['Start Date'].notnull(),
                      df['Start Date'],
                      df['Bill Date'])
df['Full Start Date'].isnull().sum()
df['Workload Month']=df['Full Start Date'].apply(lambda x:pd.date_range(start=x, periods=1, freq='M').date[0])

In [ ]:
def range_date(start,periods):
    return pd.date_range(start=start, periods=periods+1, freq='M')
df['Array Busy Months']=df.apply(lambda x: range_date(x['Full Start Date'], x['Duration Months']), axis=1)
df['Array Busy Months']=df['Array Busy Months'].apply(lambda x: x.date)

In [ ]:
df['Array Busy Months']=df['Array Busy Months'].apply(lambda x: x.flatten())

In [ ]:
depl_list=[]
month_list=[]
for i in df.index:
    month_array = df.get_value(i,'Array Busy Months')
    depl = df.get_value(i,'Deployment Consultant')
    for month in month_array:
        depl_list.append(depl)
        month_list.append(month)

In [ ]:
workload=pd.DataFrame({'Deployment Consultant':depl_list,'Workload Month':month_list})
workload.head()

In [ ]:
workload=workload.groupby(by=['Deployment Consultant','Workload Month'],as_index=False).size().reset_index(name='Workload Deployments')

In [ ]:
df=pd.merge(df,workload,how='left', on=['Deployment Consultant','Workload Month'], sort=False)

In [ ]:
df.columns

In [ ]:
df.drop(['Full Start Date', 'Workload Month', 'Array Busy Months'],inplace=True,axis=1)

In [ ]:
df['DR']=np.where((df['Project Status'].isin(['In Progress','New','Started','On Hold']))|(df['End Date']>df['Today']),1,0)

In [ ]:
import seaborn as sns
sns.boxplot(x="Package (group)", y="Duration Days", data=df,linewidth=2)

In [ ]:
dur_quantile95 = df.groupby(['Package (group)'])['Duration Days'].quantile(.95).reset_index(name='Duration Days (95 quantile)')
df=pd.merge(df,dur_quantile95,how='left', on=['Package (group)'], sort=False)

In [ ]:
df.head()

In [ ]:
columns=['Bill Number','Deployment ID','Bookings Amount','Deployment Owner','Project Status',
         'Outsource / Internal','Deployment Consultant','NA / INTL','Technical Invoice','Duration Months','Age (Months)',
        'Package Detailed','Remote / Onsite','Metal Bucket','Region','SaaS Product','Seats','MRR',
         'Account Status','Tier','Overdue','CSM Segment','DR','Workload Deployments',
         'Package (group) durations','Duration Days (95 quantile)','Overdue Days', 'Duration Overall']
dfo=df[columns]
dfo.head()

NA can be misinterpreted, so change it to North America

In [ ]:
dfo['NA / INTL']=dfo['NA / INTL'].replace('NA','North America')
dfo['NA / INTL'].unique().tolist()

In [ ]:
df['Project Status'].unique()

Filter technical invoices and error deployments

In [ ]:
dfo=dfo[dfo['Technical Invoice']!=True]
dfo=dfo[dfo['Project Status'].isin(['Error','Canceled','Cancelled']) == False]
dfo.head()

In [ ]:
dfo=dfo.drop(columns=['Technical Invoice','Metal Bucket'])

In [ ]:
dfo.isnull().sum()

In [ ]:
dfo.head()

In [ ]:
dfo[dfo['Bill Number']=='INV00366333'].T

For train drop deployments that are completed, but for test check on the completed deployments
Sekect only objects columns and drop Bill Number and Deployment ID for model training
Encode all objects values, because classifier doesn't recognize dtype objects

In [ ]:
dfo[dfo['Project Status'].isin(['Completed', 'Deemed Delivered'])].isnull().sum()

In [ ]:
dfo[dfo['Project Status'].isin(['Completed', 'Deemed Delivered']) == False].isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder
traindf=dfo[(dfo['Project Status'].isin(['Completed', 'Deemed Delivered']))&(df['Duration Months']<=6)].dropna()
#traindf=traindf[traindf['Duration Overall']<traindf['Duration Days (95 quantile)']]
testdf=dfo[dfo['Project Status'].isin(['Completed', 'Deemed Delivered']) == False].dropna()
enc = LabelEncoder()
obj_test = testdf.select_dtypes(include=['object']).copy()
obj_test = obj_test.drop(columns=['Bill Number','Deployment ID','NA / INTL','Region',
                                  'Outsource / Internal','Remote / Onsite','CSM Segment','Package Detailed',
                                  'Deployment Owner','Project Status'])

obj_test = obj_test.apply(lambda col: enc.fit_transform(col))
df_test=pd.concat([testdf[['Bill Number','Deployment ID','Bookings Amount','Overdue Days',
                           'Seats','MRR','Workload Deployments','Package (group) durations']].dropna(),obj_test],axis=1)
obj_train = traindf.select_dtypes(include=['object']).copy()
obj_train = obj_train.drop(columns=['Bill Number','Deployment ID','NA / INTL','Region',
                                  'Outsource / Internal','Remote / Onsite','CSM Segment','Package Detailed',
                                   'Deployment Owner','Project Status'])
obj_train = obj_train.apply(lambda col: enc.fit_transform(col))
df_train=pd.concat([traindf[['Bill Number','Deployment ID','Bookings Amount','Duration Months','Overdue Days',
                             'Seats','MRR','Workload Deployments','Package (group) durations']].dropna(),obj_train],axis=1)

In [ ]:
len(traindf)

In [ ]:
df_train.columns

In [ ]:
from sklearn.model_selection import train_test_split
feature_cols=['Bill Number','Deployment ID','Bookings Amount',
              'Deployment Consultant','Package (group) durations',
              'Overdue','Workload Deployments','Overdue Days',
             'SaaS Product','Seats','MRR','Account Status','Tier']
data_inputs = df_train[feature_cols]
expected_output = df_train[['Duration Months']].replace(0,0.01).astype(str)
inputs_train, inputs_test, expected_output_train, expected_output_test = train_test_split(data_inputs, expected_output, test_size=0.1, random_state=42)
inputs_train.head()

In [ ]:
sns.set(style="ticks", color_codes=True);
palette = sns.xkcd_palette(['dark blue', 'gold', 'orange'])
sns.pairplot(df_train[['Duration Months','Bookings Amount', 
                      'Seats', 'MRR', 'Workload Deployments','Overdue Days',
                      'Package (group) durations','Overdue']], hue = 'Overdue', diag_kind = 'kde', palette= palette, plot_kws=dict(alpha = 0.7),
                   diag_kws=dict(shade=True))

In [ ]:
sns.set(style="ticks", color_codes=True);
palette = sns.xkcd_palette(['dark blue', 'gold', 'orange'])
sns.pairplot(df_train[['Duration Months','Overdue','Overdue Days']], diag_kind = 'kde', palette= palette, plot_kws=dict(alpha = 0.7),
                   diag_kws=dict(shade=True))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
regressor = LinearRegression()  
X=inputs_train[['Duration Overall','Overdue']]
regressor.fit(X, expected_output_train)

In [ ]:
print(regressor.intercept_)

In [ ]:
coeff_df = pd.DataFrame(regressor.coef_.T, X.columns, columns=['Coefficient'])  
coeff_df

In [ ]:
X_test=inputs_test[['Duration Overall','Overdue']]
y_pred = regressor.predict(X_test)

In [ ]:
metrics.mean_absolute_error(np.array(expected_output_test.iloc[:,0]),y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(np.array(expected_output_train), regressor.predict(X))))
r2 = r2_score(np.array(expected_output_train), regressor.predict(X))

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
rmse = (np.sqrt(mean_squared_error(np.array(expected_output_test.iloc[:,0]), y_pred)))
r2 = r2_score(np.array(expected_output_test.iloc[:,0]), y_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

In [ ]:
df['Overdue'].replace({'T':1,'F':0},inplace=True)

In [ ]:
predict_reg=regressor.predict(df[['Duration Overall','Overdue']]).astype('int')
predict_reg=pd.DataFrame(predict_reg,columns=['Overdue Days Predict'])
final=pd.concat([df.reset_index(drop=True),predict_reg], axis=1)

In [ ]:
final.columns

In [ ]:
from datetime import timedelta
def add_days(date, days):
    return date + timedelta(days=days)
final['Predict End Date']=final.apply(lambda x: add_days(x['Start Date'], x['Package (group) durations']), axis=1)
final['Predict End Date']=final.apply(lambda x: add_days(x['Predict End Date'], x['Overdue Days Predict']), axis=1)
#final['Overdue Days']

In [ ]:
final.head()

In [ ]:
final['Predict Month']=final['Predict End Date'].apply(lambda x:x.replace(day=1))

In [ ]:
final_not_close=final[final['Project Status'].isin(['Completed', 'Deemed Delivered']) == False]

In [ ]:
final_not_close['Bookings Amount'].sum()

In [ ]:
predicted_revenue=final_not_close.pivot(columns='Predict Month', values='Bookings Amount').sum()

In [ ]:
predicted_revenue

In [ ]:
pd.DataFrame(predicted_revenue, columns = ['DR'])

In [ ]:
len(df_test.head().columns)

In [ ]:
len(df_train.head().columns)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4, 6, 8, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters

rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(inputs_train.drop(columns=['Bill Number','Deployment ID']), expected_output_train)

In [ ]:
rf_random.best_params_

In [ ]:
from sklearn import metrics
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    accuracy = metrics.accuracy_score(test_labels, predictions)
    print('Model Performance')
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(inputs_train.drop(columns=['Bill Number','Deployment ID']), 
               expected_output_train)
base_accuracy = evaluate(base_model, inputs_test.drop(columns=['Bill Number','Deployment ID']), expected_output_test.iloc[:,0])

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, inputs_test.drop(columns=['Bill Number','Deployment ID']), expected_output_test.replace(0,0.01).iloc[:,0])

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [5,10,20,30],
    'max_features': ['auto'],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [50, 100, 200]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

1 x 4 x 1 x 3 x 3 x 4 = 144 combonations for 3-folds

In [ ]:
grid_search.fit(inputs_train.drop(columns=['Bill Number','Deployment ID']), expected_output_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, inputs_test.drop(columns=['Bill Number','Deployment ID']), expected_output_test.replace(0,0.01).iloc[:,0])

In [ ]:
rf = RandomForestClassifier(n_estimators=500,min_samples_split=10,max_features='sqrt',
                            min_samples_leaf=5, max_depth=30, bootstrap=True, random_state=42)
rf.fit(inputs_train.drop(columns=['Bill Number','Deployment ID']), expected_output_train)

In [ ]:
acc = metrics.accuracy_score(expected_output_test.iloc[:,0], rf.predict(inputs_test.drop(columns=['Bill Number','Deployment ID'])))
print("Accuracy = {:.2f}%".format(acc*100))

In [ ]:
predict=pd.DataFrame(rf.predict_proba(df_test.drop(columns=['Bill Number','Deployment ID'])))

In [ ]:
final_pred=pd.concat([df_test.reset_index(drop=True),predict], axis=1)

In [ ]:
final_pred.columns

In [ ]:
for i in range(1,7):
    final_pred[str(i)+'_pred']=final_pred.loc[:,i]*final_pred['Bookings Amount']

In [ ]:
final_pred

In [ ]:
final_pred['Bookings Amount'].sum()

In [ ]:
final_pred[final_pred['Overdue Days']>180]

In [ ]:
final_pred=final_pred.merge(df[['Bill Number','Deployment ID','Bookings Amount','Start Date','Bill Date']],how='left', on=['Bill Number','Deployment ID','Bookings Amount'])

In [ ]:
from datetime import timedelta
def add_month(date, month):
    date = date.replace(day=1)
    date = date + month*timedelta(days=32)
    date = date.replace(day=1)
    return date
final_pred['Full Start Date'] = np.where(final_pred['Start Date'].notnull(),
                      final_pred['Start Date'],
                      final_pred['Bill Date'])
final_pred['Next Month']=final_pred.apply(lambda x: add_month(x['Full Start Date'], 1), axis=1)

In [ ]:
final_pred.groupby(['Next Month'])['1_pred'].sum()

In [ ]:
bins = np.linspace(300, 600, 50)

plt.hist(inputs_train['Duration Overall'], bins, alpha=0.5, label='train')
plt.hist(df_test['Duration Overall'], bins, alpha=0.5, label='real')
plt.legend(prop={'size': 10})
plt.show()

In [ ]:
bins = np.linspace(0, 600, 50)

plt.hist(inputs_train['Duration Overall'], bins, alpha=0.5, label='train')
plt.hist(df_test['Duration Overall'], bins, alpha=0.5, label='real')
plt.legend(prop={'size': 10})
plt.show()

In [ ]:
predict_array=rf.predict(df_test.drop(columns=['Bill Number','Deployment ID'])).astype('int')
predict=pd.DataFrame(predict_array,columns=['Overdue Days'])
final=pd.concat([df_test.reset_index(drop=True),predict], axis=1)

In [ ]:
rf.predict(df_test.drop(columns=['Bill Number','Deployment ID'])).astype('int')

In [ ]:
final

In [ ]:
final.to_csv(r'G:\My Drive\PS Model Automation\File Name.csv',index=False)

In [ ]:
pd.DataFrame({'feature': data_inputs.drop(columns=['Bill Number','Deployment ID']).columns,
              'importance': rf.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
# import xgboost
# from sklearn import model_selection
# from sklearn.metrics import accuracy_score
# model = xgboost.XGBClassifier()
# model.fit(inputs_train.drop(columns=['Bill Number','Deployment ID']), expected_output_train)
# print(model)
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
# # evaluate predictions
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# # make predictions for test data
# y_pred = model.predict(inputs_test.drop(columns=['Bill Number','Deployment ID']))
# predictions = [round(value) for value in y_pred]
# # evaluate predictions
# accuracy = accuracy_score(expected_output_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# pd.DataFrame({'feature': data_inputs.drop(columns=['Bill Number','Deployment ID']).columns,
#               'importance': model.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
traindf=dfo[dfo['Project Status'].isin(['Completed', 'Deemed Delivered'])].dropna()
traindf=traindf[traindf['Duration Days']<traindf['Duration Days (95 quantile)']]
testdf=dfo[dfo['Project Status'].isin(['Completed', 'Deemed Delivered']) == False].drop('Duration Days',axis=1).dropna()

feature_cols=['Bill Number','Deployment ID','Bookings Amount','Deployment Owner','Project Status',
              'Deployment Consultant','Package (group) durations',
              'Package Detailed','Overdue','Workload Deployments','Account Industry',
             'SaaS Product','Seats','MRR','Account Status','Tier','Days from Today']
data_inputs = traindf[feature_cols]
expected_output = traindf[['Duration Days']]
inputs_train, inputs_test, expected_output_train, expected_output_test = train_test_split(data_inputs, expected_output, test_size=0.1, random_state=42)
inputs_train.head()

In [ ]:
from sklearn import preprocessing
from catboost import CatBoostClassifier, FeaturesData, Pool

le=preprocessing.LabelEncoder()
expected_output = le.fit_transform(expected_output_train)
feature_num=['Bookings Amount','Seats','MRR','Days from Today',
             'Workload Deployments','Package (group) durations']
feature_cat=['Deployment Owner', 'Project Status',
       'Deployment Consultant',
       'Package Detailed', 'SaaS Product',
       'Account Status', 'Tier', 'Overdue', 'Account Industry']
inputs_train[feature_num]=inputs_train[feature_num].astype('float32')
inputs_test[feature_num]=inputs_test[feature_num].astype('float32')
inputs_train[feature_cat]=inputs_train[feature_cat].astype('str')
inputs_test[feature_cat]=inputs_test[feature_cat].astype('str')
data=FeaturesData(
        num_feature_data=inputs_train[feature_num].values,
        cat_feature_data=inputs_train[feature_cat].values,
        num_feature_names=feature_num,
        cat_feature_names=feature_cat)

train_data = Pool(
    data=FeaturesData(
        num_feature_data=inputs_train[feature_num].values,
        cat_feature_data=inputs_train[feature_cat].values,
        num_feature_names=feature_num,
        cat_feature_names=feature_cat),
    label=expected_output_train
)
test_data = Pool(
    data=FeaturesData(
        num_feature_data=inputs_test[feature_num].values,
        cat_feature_data=inputs_test[feature_cat].values,
        num_feature_names=feature_num,
        cat_feature_names=feature_cat),
    label=expected_output_test
)

In [ ]:
model = CatBoostRegressor(iterations=800, learning_rate=1, depth=10)
# Fit model
model.fit(train_data,plot=True,eval_set=test_data)
# Get predicted classes
preds_class = model.predict(test_data)

In [ ]:
print ("test_score: {0:.2f}, train_score: {1:.2f}".format(model.score(test_data),model.score(train_data)))

In [ ]:
print(round(model.eval_metrics(test_data, metrics=['R2'])['R2'][-1],4))

In [ ]:
pd.DataFrame({'feature': model.feature_names_,
              'importance': np.round(model.get_feature_importance(train_data),1)}).sort_values('importance', ascending=False)

In [ ]:
# cat_model.score(inputs_test.drop(columns=['Bill Number','Deployment ID']), expected_output_test)

In [ ]:
# pd.DataFrame({'feature': data_inputs.drop(columns=['Bill Number','Deployment ID']).columns,
#               'importance': cat_model.get_feature_importance()}).sort_values('importance', ascending=False)